<a href="https://colab.research.google.com/github/xinformatics/streamlit_workshop/blob/main/02_Workshop_Feb_05_2024_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!npm install localtunnel
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.925s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit 

In [2]:
!chmod +x cloudflared-linux-amd64
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

nohup: appending output to 'nohup.out'


In [6]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://charger-pharmacology-incorporated-plan.trycloudflare.com


In [4]:
%%writefile app.py
import streamlit as st

# Title of the application
st.title('My First ML model deployment')

Writing app.py


In [5]:
!streamlit run /content/app.py &>/content/logs.txt &

## once it's running you can keep adding text without shuting the app down.



# **Computer Vision Example- Building an Image Classifier with InceptionV3**



In [11]:
%%writefile app.py


from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import streamlit as st

# Initialize the InceptionV3 model
model = InceptionV3(weights='imagenet')


def classify_image(img):
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    preds = model.predict(img)
    return decode_predictions(preds, top=10)[0]

def main():
    st.title("Image Classification with InceptionV3")
    st.write("Upload an image and the model will classify it into the top 10 categories.")

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        img = image.load_img(uploaded_file, target_size=(299, 299))
        st.image(img, caption='Uploaded Image', use_column_width=False)
        st.write("")
        st.write("Classifying...")
        labels = classify_image(img)
        for label in labels:
            st.write(f"{label[1]} ({label[2]*100:.2f}%)")
            st.progress(int(label[2] * 100))

if __name__ == '__main__':
    main()

Overwriting app.py


# **Computer Vision Example - Demo**

In [19]:
%%writefile app.py

import streamlit as st
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
import numpy as np
import colorsys

# Define COCO labels mapping
COCO_LABELS = {
    1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane',    6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light',
    11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench',    16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep',
    21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe',    27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie',
    33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',    37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
    41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',    46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
    51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',    56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
    61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',    67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
    75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave',    79: 'oven', 80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book',
    85: 'clock', 86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier',    90: 'toothbrush'}


def get_color(label_id):
    # Generate a unique color for each label id based on its hash value
    hue = hash(label_id) % 360  # Limit hue to [0, 360]
    saturation = 90 + hash(label_id) % 10  # Saturation between [90, 100]
    lightness = 50 + hash(label_id) % 10  # Lightness between [50, 60]

    color = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)
    # Convert to RGB format as integer tuple
    return tuple(int(c * 255) for c in color)

@st.cache_resource()
def load_model():
    model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    model.eval()
    return model

model = load_model()

def object_detection(image, model):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        prediction = model(image)

    return prediction[0]

def main():
    st.title("Object Detection with FasterRCNN")

    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert('RGB')
        st.image(image, caption='Uploaded Image', use_column_width=True)

        prediction = object_detection(image, model)

        # Prepare to draw
        draw = ImageDraw.Draw(image)
        # Define font for drawing text
        font_size = 20
        font = ImageFont.load_default()


        for element in range(len(prediction["boxes"])):
            boxes = prediction["boxes"][element].cpu().numpy()
            score = np.round(prediction["scores"][element].cpu().numpy(), decimals=4)
            label_id = int(prediction["labels"][element].cpu().numpy())
            label_name = COCO_LABELS.get(label_id, 'N/A')
            color = get_color(label_id)
            text_color = (255, 255, 255)  # White color for text

            if score > 0.5:
                label_with_score = f"{label_name}: {score:.2f}"
                draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], outline=color, width=3)
                draw.text((boxes[0], boxes[1] - 10), label_with_score, fill=text_color, font=font)

        st.image(image, caption="Processed Image", use_column_width=True)

if __name__ == '__main__':
    main()





Overwriting app.py


# **Natural Language Processing - Emotion Detection**